In [2]:
import pandas as pd
import sqlite3
# import plotly.express as px
# import plotly.graph_objects as go
import numpy as np

In [3]:
con = sqlite3.connect('prod.db')

In [57]:
df = pd.read_sql_query("""
SELECT l.match_id
,  l.account_id
,  fp.name
, dp.role
, l.kills
, l.deaths
,l.assists
, l.last_hits
, l.denies
, l.gold_per_min
, l.teamfight_participation
, l.stuns
, l.tower_kills
, l.roshan_kills
, l.firstblood_claimed
, l.camps_stacked
, l.rune_pickups
, l.obs_placed
from logs_player_match_stats as l 
JOIN dim_players as dp
ON dp.account_id = l.account_id
JOIN f_players as fp
ON l.account_id = fp.account_id
/* WHERE fp.team_tag in ('Alliance', 'OG','EG', 'UND', 'IG', 'VP') */
""",
 con).fillna(0)
# 

In [58]:
def make_fantasy_points(df):
    df['FP'] = df['kills']*.3 \
                    + 3 + df['deaths']*-.3 \
                    + (df['last_hits']+df['denies'])*0.003\
                    + df['gold_per_min']*0.002\
                    + df['teamfight_participation']*3\
                    + df['stuns']*0.05\
                    + df['tower_kills'] \
                    + df['roshan_kills'] \
                    + df['firstblood_claimed']*4 \
                    + df['camps_stacked']*0.5 \
                    + df['rune_pickups']*0.25 \
                    + df['obs_placed']*0.5
    for col in df.columns[4:]:
        df[f'{col}_rolling'] = df[col].rolling(window=5).mean()
        df[f'{col}_final'] = df[f'{col}_rolling'].tail(1).values[0]
    return df

In [59]:
df = df.groupby('account_id').apply(make_fantasy_points)

In [60]:
df

,match_id,account_id,name,role,kills,deaths,assists,last_hits,denies,gold_per_min,...,firstblood_claimed_rolling,firstblood_claimed_final,camps_stacked_rolling,camps_stacked_final,rune_pickups_rolling,rune_pickups_final,obs_placed_rolling,obs_placed_final,FP_rolling,FP_final
0,6212505407,138857296,JT-,3,1,2,15,240,9,468,...,NaN,0.2,NaN,1.8,NaN,3.8,NaN,0.0,NaN,13.338577
1,6212505407,168028715,flyfly,1,10,0,6,378,18,690,...,NaN,0.2,NaN,0.6,NaN,2.6,NaN,0.2,NaN,17.294697
2,6212505407,139876032,kaka,4,2,5,8,62,1,262,...,NaN,0.0,NaN,4.0,NaN,2.2,NaN,8.0,NaN,13.470715
3,6212505407,202217968,Emo,2,7,1,9,331,15,594,...,NaN,0.0,NaN,2.2,NaN,17.0,NaN,0.4,NaN,18.114034
4,6212505407,184950344,Ku,3,2,5,5,166,23,341,...,NaN,0.2,NaN,1.0,NaN,2.2,NaN,0.0,NaN,9.683024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,6214092702,124801257,Nightfall,1,5,4,2,367,10,565,...,0.0,0.0,1.6,1.6,5.4,5.4,0.0,0.0,11.186120,11.186120
712,6214092702,480412663,gpk~,2,3,5,7,268,7,453,...,0.2,0.2,0.8,0.8,7.6,7.6,1.8,1.8,12.988347,12.988347
713,6214092702,56351509,DM,3,1,8,5,125,3,294,...,0.2,0.2,0.4,0.4,2.8,2.8,0.0,0.0,10.496890,10.496890
714,6214092702,317880638,Save-,4,4,8,7,83,1,273,...,0.0,0.0,1.2,1.2,2.6,2.6,5.8,5.8,10.572108,10.572108


In [61]:
def drop_cols(df):
    for col in df.columns[4:]:
        if not col.endswith('_final'):
            df.drop(col, axis = 1, inplace = True)
    return df

In [62]:
df = drop_cols(df)

In [63]:
df.drop('match_id', axis = 1, inplace = True)

In [64]:
df.drop_duplicates(inplace = True)

In [65]:
df.reset_index(inplace=True, drop = True)

In [66]:
df

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
0,138857296,JT-,3,7.2,2.0,11.4,282.6,6.8,636.8,0.640651,21.296488,0.6,0.4,0.2,1.8,3.8,0.0,13.338577
1,168028715,flyfly,1,10.0,1.6,7.2,348.2,16.4,657.6,0.594883,10.620940,4.2,1.0,0.2,0.6,2.6,0.2,17.294697
2,139876032,kaka,4,2.2,4.8,14.8,90.6,2.0,322.4,0.585742,40.417781,0.0,0.0,0.0,4.0,2.2,8.0,13.470715
3,202217968,Emo,2,8.2,2.6,10.6,367.2,11.0,665.6,0.652331,21.224810,2.2,0.2,0.0,2.2,17.0,0.4,18.114034
4,184950344,Ku,3,5.8,6.0,12.8,162.8,15.8,439.2,0.639028,27.234816,0.2,0.0,0.2,1.0,2.2,0.0,9.683024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,117956848,tavo,3,5.4,5.8,11.6,171.2,6.8,374.8,0.684986,35.824894,0.4,0.0,0.0,0.4,3.2,0.0,9.409802
85,85937380,4dr ♥ Deia,2,7.4,3.6,8.6,343.0,6.0,529.6,0.663161,29.669117,1.2,0.8,0.4,0.6,7.2,0.6,15.719140
86,105045291,Thiolicor,4,2.4,8.4,10.4,109.8,2.2,297.0,0.548844,22.285046,0.6,0.0,0.2,2.4,4.2,5.2,11.140784
87,86822085,Costabile,1,6.8,4.8,8.2,459.8,6.4,632.4,0.593380,39.874296,2.2,0.8,0.0,1.0,6.8,0.0,15.237254


In [67]:
df.sort_values(by = ['role', 'FP_final'])

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
79,95825708,MNZ,1,4.6,5.6,3.6,371.2,10.2,566.8,0.482838,3.785791,1.0,0.0,0.2,2.4,2.4,0.0,10.215603
24,124801257,Nightfall,1,4.6,4.8,10.4,375.6,8.4,572.2,0.623172,18.604054,0.6,0.4,0.0,1.6,5.4,0.0,11.186120
8,375507918,23,1,6.6,4.6,12.0,320.0,11.8,604.0,0.659028,13.623998,0.8,0.4,0.2,2.2,2.2,0.0,12.111683
63,108452107,YS,1,4.6,2.6,5.0,385.6,11.4,623.2,0.518741,24.571948,1.6,0.4,0.0,1.4,3.8,0.0,12.472221
48,137193239,eurus,1,4.2,4.0,2.4,342.0,10.0,573.4,0.507586,21.173503,1.8,0.2,0.2,2.6,3.0,0.0,12.694234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,138543123,Pyw,4,5.4,3.6,16.6,138.6,5.4,382.6,0.720429,22.124272,0.4,0.6,0.0,2.0,5.8,7.2,15.054700
83,81306398,KJ,4,1.8,7.8,12.2,26.8,3.0,241.0,0.578397,33.123294,0.0,0.0,0.0,2.8,3.8,15.2,15.112756
44,102099826,DJ,4,2.4,7.0,11.6,85.0,3.0,312.2,0.602500,7.673786,0.2,0.0,0.2,3.0,2.8,15.8,15.799589
69,89117038,YapzOr,4,6.0,3.8,15.6,74.6,2.4,335.2,0.705292,40.776368,0.2,0.0,0.0,1.0,4.6,14.6,17.866093


In [68]:
df_core = df[df['role'].isin(['1','3']) ]

In [69]:
df_mid  = df[df['role'] == '2' ]

In [70]:
df_sup = df[df['role'] == '4' ]

In [71]:
df_core.sort_values(by = 'FP_final', ascending=False)

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
34,148215639,Monet,1,10.6,2.2,9.6,395.2,9.8,693.2,0.740375,23.565136,3.6,1.2,0.4,0.4,4.4,0.0,19.220781
10,111620041,SumaiL,1,9.6,4.2,5.6,393.6,11.2,648.2,0.608344,31.562291,4.0,0.6,0.2,1.2,7.8,0.4,18.683946
38,277141004,poyoyo,1,7.4,1.2,13.8,505.0,15.4,716.6,0.694520,5.953632,5.0,0.6,0.2,1.6,4.6,0.0,18.585643
23,97658618,Timado,1,10.0,1.4,14.8,426.0,10.4,723.6,0.676272,0.981973,4.6,0.6,0.0,2.4,6.2,0.2,18.464314
46,132309493,Raven,1,8.6,4.0,6.4,521.6,7.2,675.4,0.658214,28.308051,3.4,1.2,0.0,2.2,5.6,0.2,17.907245
1,168028715,flyfly,1,10.0,1.6,7.2,348.2,16.4,657.6,0.594883,10.620940,4.2,1.0,0.2,0.6,2.6,0.2,17.294697
58,898754153,萧瑟,1,5.8,3.0,9.4,467.4,10.6,777.4,0.587217,25.044746,4.6,0.6,0.0,2.0,4.6,0.0,17.192688
56,164685175,k1,1,7.0,3.2,10.4,441.8,10.0,734.8,0.653281,18.402970,4.6,0.4,0.2,0.6,4.2,0.0,16.994993
72,72312627,MATUMBAMAN,1,8.2,5.4,11.6,403.0,11.6,657.8,0.614033,34.982477,2.8,0.4,0.2,2.0,3.6,0.0,15.890624
29,86745912,Arteezy,1,5.8,2.8,4.8,316.4,13.6,595.6,0.565029,22.735575,3.2,0.6,0.2,2.2,2.2,1.0,15.663067


In [72]:
df_mid.sort_values(by = 'FP_final', ascending=False)

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
20,238239590,Bryle,2,12.8,1.6,12.0,392.4,10.0,678.8,0.718142,39.311564,2.4,1.0,0.4,2.2,11.2,0.4,22.144803
40,107803494,曾焦阳,2,9.0,3.0,13.0,363.6,11.6,598.2,0.715601,21.922312,2.2,0.6,0.0,2.0,17.4,0.0,18.514918
3,202217968,Emo,2,8.2,2.6,10.6,367.2,11.0,665.6,0.652331,21.224810,2.2,0.2,0.0,2.2,17.0,0.4,18.114034
70,121769650,Nisha,2,8.4,3.8,12.0,366.0,6.4,601.8,0.698395,15.630802,2.2,0.6,0.0,4.4,9.4,1.2,17.527525
54,153836240,Chris luck,2,6.2,2.6,11.8,324.2,12.4,589.0,0.680533,18.428351,1.2,0.4,0.2,2.4,14.8,1.2,17.130815
5,94281932,Karl,2,9.4,2.6,9.0,236.4,13.6,563.2,0.652222,33.326370,1.4,0.0,0.0,2.0,11.4,1.8,16.689385
85,85937380,4dr ♥ Deia,2,7.4,3.6,8.6,343.0,6.0,529.6,0.663161,29.669117,1.2,0.8,0.4,0.6,7.2,0.6,15.719140
59,173978074,NothingToSay,2,8.8,3.4,8.8,354.6,7.4,642.4,0.688556,27.587456,1.0,0.0,0.0,2.2,7.6,1.0,14.935840
30,154715080,Abed,2,3.8,3.0,8.4,282.6,9.2,522.0,0.675331,24.021971,0.6,0.0,0.0,3.0,13.8,1.8,14.836491
47,113435203,ChYuaN,2,4.6,5.2,11.2,407.2,3.8,548.2,0.657143,42.579242,1.0,0.2,0.2,1.8,8.0,1.2,14.749791


In [56]:
df_sup.sort_values(by = 'FP_final', ascending=False)

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
69,89117038,YapzOr,4,4.875000,4.125000,15.125000,58.625000,2.250000,318.750000,0.685542,38.374805,0.250000,0.000000,0.125000,1.250000,3.750000,13.500000,17.082991
44,102099826,DJ,4,2.000000,5.875000,14.000000,70.750000,2.000000,301.500000,0.644405,20.781445,0.125000,0.000000,0.125000,3.250000,3.750000,16.375000,17.006038
42,143693439,Dy,4,2.250000,4.250000,19.875000,70.375000,3.875000,336.250000,0.761724,47.196973,0.000000,0.000000,0.250000,2.750000,2.875000,11.625000,16.846520
2,139876032,kaka,4,2.300000,5.800000,13.900000,76.600000,1.900000,301.300000,0.575989,53.978201,0.100000,0.000000,0.200000,3.600000,2.500000,9.200000,15.139976
83,81306398,KJ,4,1.857143,7.857143,11.428571,29.285714,2.428571,246.857143,0.615522,31.236567,0.000000,0.142857,0.000000,2.285714,3.571429,15.142857,14.947251
41,138543123,Pyw,4,4.000000,4.000000,15.875000,131.750000,4.500000,365.875000,0.687478,22.365420,0.250000,0.500000,0.125000,1.875000,5.375000,8.000000,14.852456
22,145550466,DuBu,4,2.500000,6.625000,15.250000,22.625000,1.875000,281.875000,0.640031,34.133858,0.000000,0.000000,0.000000,2.500000,1.750000,13.875000,14.651537
55,119631156,ST!NG3R,4,1.857143,7.571429,13.571429,43.285714,2.285714,287.142857,0.677344,25.081350,0.142857,0.000000,0.000000,6.000000,1.714286,11.142857,14.425672
6,136829091,Whitemon,4,2.125000,5.250000,13.625000,25.750000,5.500000,235.750000,0.691814,47.546471,0.000000,0.000000,0.000000,2.125000,2.000000,11.500000,14.393014
12,103735745,Saksa,4,3.600000,7.100000,13.900000,80.400000,4.300000,332.000000,0.634378,46.836145,0.200000,0.000000,0.100000,3.100000,4.600000,7.700000,14.263040
